In [1]:
# importing libraries
import os
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split

In [2]:
PROJECT_DIR = r"/Users/harsimranjitsingh/Desktop/Projects"
DATA_DIR = "data"

In [3]:
def get_data(name):
    file_name = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR, DATA_DIR, file_name)
    return pd.read_csv(file_path)
    

In [4]:
# Reading the Data
flights = get_data("flight_price")

In [5]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


dataset contains 10,683 rows and 11 features
"route" and "total_stops" have missing value each
the datatypes of features not appropriate

In [6]:
## Preliminary analysis

In [7]:
# Check the datatypes 
flights.head()


,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [8]:
flights.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

In [9]:
# check for duplicates

flights.duplicated().sum()


220

In [10]:
(
    flights.loc[flights.duplicated(keep=False)]
    .sort_values(['Airline', 'Date_of_Journey','Source','Destination'])
)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
572,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
8168,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
1495,Air India,1/04/2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,01:20 02 Apr,15h 20m,2 stops,No info,10408
...,...,...,...,...,...,...,...,...,...,...,...
2692,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
2870,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
3711,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
2634,Vistara,24/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,5403


#observations 
- The type of Date_of_journey, Dep_time , Arrival_time should be date type
- The type of duration and total_stops is mixed it should be numeric
- There are 220 duplicates these should be removed

In [11]:
# Detailed analyssi

In [12]:
flights.Dep_Time

(
    flights
    .Dep_Time
    .loc[lambda ser: ser.str.contains("[^0-9:]")]
)
pd.to_datetime(flights.Dep_Time).dt.time

/var/folders/6v/l32qpxw579j25nxjzdxgdqdh0000gn/T/ipykernel_86698/590031785.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(flights.Dep_Time).dt.time


0        22:20:00
1        05:50:00
2        09:25:00
3        18:05:00
4        16:50:00
           ...   
10678    19:55:00
10679    20:45:00
10680    08:20:00
10681    11:30:00
10682    10:55:00
Name: Dep_Time, Length: 10683, dtype: object

In [13]:
# Arrival time 

(
    flights
    .Arrival_Time
    .loc[lambda ser: ser.str.contains("[^0-9:]")]
    .str.split(" ", n=1)
    .str.get(1)
    .unique()
)

# duration

flights.Duration

0        2h 50m
1        7h 25m
2           19h
3        5h 25m
4        4h 45m
          ...  
10678    2h 30m
10679    2h 35m
10680        3h
10681    2h 40m
10682    8h 20m
Name: Duration, Length: 10683, dtype: object

In [14]:
(
    flights
    .Duration
    .loc[lambda ser: ~ser.str.contains("m")]
    .unique()
)
(
    flights
    .Duration
    .loc[lambda ser: ~ser.str.contains("h")]

)
flights.iloc[[6474]]

(
    flights
    .Duration
    .drop(index=6474)
    .str.split(" ", expand=True)
    .set_axis(["hour", "minute"], axis=1)
    .assign(
        hour= lambda df_ : (
            df_
            .hour
            .str.replace("h", "")
            .astype(int)
            .mul(60)
        ),
        minute = lambda df_ :(
            df_
            .minute
            .str.replace("m", "")
            .fillna("0")
            .astype(int)
        )
    )
    .sum(axis=1)
    
)

## delete 6474 has duration of 5 minutes we will delete this observations


0         170
1         445
2        1140
3         325
4         285
         ... 
10678     150
10679     155
10680     180
10681     160
10682     500
Length: 10682, dtype: int64

In [15]:
(
    flights
    .Duration
    .drop(index=6474)
    .str.split(" ", expand=True)
    .set_axis(["hour", "minute"], axis=1)
    .assign(
        hour= lambda df_ : (
            df_
            .hour
            .str.replace("h", "")
            .astype(int)
            .mul(60)
        ),
        minute = lambda df_ :(
            df_
            .minute
            .str.replace("m", "")
            .fillna("0")
            .astype(int)
        )
    )
    .sum(axis=1)
    .rename("duration_minutes")
    .to_frame()
    .join(flights.Duration.drop(index=6474))
)
flights.Additional_Info.unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

In [16]:
def convert_to_minutes(ser):
    return (
        ser
        .str.split(" ", expand=True)
        .set_axis(["hour", "minute"], axis=1)
    .   assign(
            hour= lambda df_ : (
                df_
                .hour
                .str.replace("h", "")
                .astype(int)
                .mul(60)
        ),
        minute = lambda df_ :(
            df_
            .minute
            .str.replace("m", "")
            .fillna("0")
            .astype(int)
        )
    )
    .sum(axis=1)
    )

In [17]:
flights.Total_Stops.unique()

(
    flights
    .Total_Stops
    .replace("non-stop", "0")
    .str.replace(" stops?", "", regex=True)
    .pipe(lambda ser: pd.to_numeric(ser))
)

0        0.0
1        2.0
2        2.0
3        1.0
4        1.0
        ... 
10678    0.0
10679    0.0
10680    0.0
10681    0.0
10682    2.0
Name: Total_Stops, Length: 10683, dtype: float64

In [18]:
def convert_to_minutes(ser):
    return (
        ser
        .str.split(" ", expand=True)
        .set_axis(["hour", "minute"], axis=1)
    .   assign(
            hour= lambda df_ : (
                df_
                .hour
                .str.replace("h", "")
                .astype(int)
                .mul(60)
        ),
        minute = lambda df_ :(
            df_
            .minute
            .str.replace("m", "")
            .fillna("0")
            .astype(int)
        )
    )
    .sum(axis=1)
    )
def clean_data(df):
    return (
        df
        .drop(index=[6474])
        .drop_duplicates()
        .assign(**{
            col: df[col].str.strip()
            for col in df.select_dtypes(include='O').columns
        })
        .rename(columns=str.lower)
        # .assign( # airline=lambda df_: df_.airline.str.lower() # use function for the current state 
        .assign(
            airline=lambda df_: (
                df_
                .airline
                .str.replace(" Premium economy", "")
                .str.replace(" Business", "")
                .str.title()
            ),
            date_of_journey= lambda df_ : pd.to_datetime(df_.date_of_journey,dayfirst=True),
            dep_time=lambda df_ : pd.to_datetime(df_.dep_time).dt.time,
            arrival_time=lambda df_ : pd.to_datetime(df_.arrival_time).dt.time,
            duration = lambda df_ : df_.duration.pipe(convert_to_minutes),
            total_stops= lambda df_ : (
                df_
                .total_stops
                .replace("non-stop", "0")
                .str.replace(" stops?", "", regex=True)
                .pipe(lambda ser: pd.to_numeric(ser))
            ),
            additional_info= lambda df_ : df_.additional_info.replace("No info" , "No Info")
        )
        .drop(columns='route')

    )

In [19]:
flights_cleaned = clean_data(flights)
flights_cleaned

/var/folders/6v/l32qpxw579j25nxjzdxgdqdh0000gn/T/ipykernel_86698/902085013.py:44: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dep_time=lambda df_ : pd.to_datetime(df_.dep_time).dt.time,
/var/folders/6v/l32qpxw579j25nxjzdxgdqdh0000gn/T/ipykernel_86698/902085013.py:45: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  arrival_time=lambda df_ : pd.to_datetime(df_.arrival_time).dt.time,


,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,Banglore,New Delhi,22:20:00,01:10:00,170,0.0,No Info,3897
1,Air India,2019-05-01,Kolkata,Banglore,05:50:00,13:15:00,445,2.0,No Info,7662
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25:00,1140,2.0,No Info,13882
3,Indigo,2019-05-12,Kolkata,Banglore,18:05:00,23:30:00,325,1.0,No Info,6218
4,Indigo,2019-03-01,Banglore,New Delhi,16:50:00,21:35:00,285,1.0,No Info,13302
...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,2019-04-09,Kolkata,Banglore,19:55:00,22:25:00,150,0.0,No Info,4107
10679,Air India,2019-04-27,Kolkata,Banglore,20:45:00,23:20:00,155,0.0,No Info,4145
10680,Jet Airways,2019-04-27,Banglore,Delhi,08:20:00,11:20:00,180,0.0,No Info,7229
10681,Vistara,2019-03-01,Banglore,New Delhi,11:30:00,14:10:00,160,0.0,No Info,12648


In [20]:
flights.Date_of_Journey

pd.to_datetime(flights.Date_of_Journey, dayfirst=True)

0       2019-03-24
1       2019-05-01
2       2019-06-09
3       2019-05-12
4       2019-03-01
           ...    
10678   2019-04-09
10679   2019-04-27
10680   2019-04-27
10681   2019-03-01
10682   2019-05-09
Name: Date_of_Journey, Length: 10683, dtype: datetime64[ns]

In [21]:
flights.Destination.unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

In [22]:
flights.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [23]:
# some of the entries have inconsistant values
(
    flights
    .Airline
    .str.replace(" Premium economy", "")
    .str.replace(" Business", "")
    .str.title() # make the 
)

0             Indigo
1          Air India
2        Jet Airways
3             Indigo
4             Indigo
            ...     
10678       Air Asia
10679      Air India
10680    Jet Airways
10681        Vistara
10682      Air India
Name: Airline, Length: 10683, dtype: object

In [24]:
# split data 

flights_final = flights_cleaned.sample(10000)


In [25]:
X = flights_final.drop(columns="price")
y= flights_final.price.copy()

In [26]:
x_ , x_test, y_, y_test = train_test_split(X, y, test_size=0.2 , random_state=42)

In [27]:
x_train, x_val, y_train, y_val = train_test_split(x_, y_, test_size=0.2 , random_state=42)

In [28]:
#export the data 

In [29]:
def export_data(X, y , name):
    file_name = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR, DATA_DIR, file_name)
    X.join(y).to_csv(file_path, index = False)

    return pd.read_csv(file_path).head()

In [30]:
export_data(x_train, y_train, "train")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-03-06,Delhi,Cochin,18:15:00,19:45:00,1530,1.0,No Info,17024
1,Indigo,2019-03-09,Delhi,Cochin,04:55:00,12:00:00,425,1.0,No Info,6171
2,Air India,2019-06-15,Delhi,Cochin,13:20:00,07:40:00,1100,2.0,No Info,12121
3,Multiple Carriers,2019-03-27,Delhi,Cochin,10:20:00,01:35:00,915,1.0,No Info,6427
4,Jet Airways,2019-05-01,Kolkata,Banglore,21:10:00,04:40:00,450,1.0,In-flight meal not included,10844


In [31]:
export_data(x_test, y_test,"test")
export_data(x_val, y_val, "val")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-27,Delhi,Cochin,06:40:00,21:00:00,860,1.0,No Info,8784
1,Jet Airways,2019-05-06,Kolkata,Banglore,20:00:00,08:15:00,735,1.0,No Info,14388
2,Jet Airways,2019-03-09,Delhi,Cochin,15:05:00,18:50:00,1665,1.0,No Info,16289
3,Jet Airways,2019-04-15,Banglore,Delhi,18:55:00,22:00:00,185,0.0,No Info,7229
4,Jet Airways,2019-06-09,Delhi,Cochin,06:45:00,12:35:00,1790,2.0,No Info,13376
